In [ ]:
# Import necessary libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
import wget
import os
import re
import requests
import pandas as pd
import matplotlib.pyplot as plt
import time

from splinter import Browser
from selenium.webdriver.firefox.service import Service


In [ ]:
def create_facebook_marketplace_query(params):
    '''Returns a URL for Facebook Marketplace given user parameters'''
    # check for the two required parameters, if they are not specified, then exit
    if 'location' not in params or 'item' not in params:
        raise ValueError('location or item was not specified.')
    
    # make default params if they are not specified
    if 'condition' not in params:
        params['condition'] = []
    
    if 'min_price' not in params:
        params['min_price'] = 0
    
    if 'max_price' not in params:
        params['max_price'] = 1000
    
    if 'days_since_listed' not in params:
        params['days_since_listed'] = 10

    # lookup table for conditions 
    condition_separator = "%2C"
    conditions = {0: "new", 1: "used_like_new", 2: "used_good", 3: "used_fair"}

    # create the conditions part of url
    condition = "" if params['condition'] == [] else "&itemCondition=" + conditions[params['condition'][0]]
    for index in params['condition']:
        condition += condition_separator + conditions[index]

    # create the query string
    base_url = f"https://www.facebook.com/marketplace/{params['location']}/search?"
    url = f"{base_url}minPrice={params['min_price']}&maxPrice={params['max_price']}&daysSinceListed={params['days_since_listed']}{condition}&query={params['item']}"
    return url

In [ ]:
def fetch_marketplace_html(url):
    """Returns Facebook Marketplace Data in the format of BeautifulSoup Object"""
    browser = Browser('firefox')
    browser.visit(url)
    # close pop out window
    if browser.is_element_present_by_css('div[aria-label="Close"]', wait_time=10):
        # Click on the element once it's found
        browser.find_by_css('div[aria-label="Close"]').first.click()
    
    # Scroll down to load more results
    
    # Define the number of times to scroll the page
    scroll_count = 4

    # Define the delay (in seconds) between each scroll
    scroll_delay = 2

    # Loop to perform scrolling
    for _ in range(scroll_count):
        # Execute JavaScript to scroll to the bottom of the page
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Pause for a moment to allow the content to load
        time.sleep(scroll_delay)
    
    # Parse the HTML
    html = browser.html

    # close browser window
    time.sleep(1)
    browser.quit()

    # Create a BeautifulSoup object from the scraped HTML
    return soup(html, 'html.parser')

In [ ]:
def parse_marketplace_data(market_soup):
    '''Returns a Pandas Data Frame with information about Facebook Marketplace listings'''

    # get listings

    listing_divs = market_soup.find_all('div', class_="x9f619 x78zum5 x1r8uery xdt5ytf x1iyjqo2 xs83m0k x1e558r4 x150jy0e x1iorvi4 xjkvuk6 xnpuxes x291uyu x1uepa24")
    data = []
    for listing_div in listing_divs:
        
        # Extract all the necessary info and insert into lists
        picture_div = listing_div.find('img', class_="xt7dq6l xl1xv1r x6ikm8r x10wlt62 xh8yej3")
        if picture_div == None:
            continue
        picture_url = picture_div.get('src')

        title_div = listing_div.find('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")
        if title_div == None:
            continue
        title = title_div.text.strip()

        price_div = listing_div.find('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x676frb x1lkfr7t x1lbecb7 x1s688f xzsf02u")
        if price_div == None:
            continue
        price = price_div.text.strip()

        location_div = listing_div.find('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x3x7a5m x1nxh6w3 x1sibtaa xo1l8bm xi81zsa")
        if location_div == None:
            continue
        location = location_div.text.strip()

        url_div = listing_div.find('a', class_="x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1sur9pj xkrqix3 x1lku1pv")
        if url_div == None:
            continue
        url = "www.facebook.com" + url_div.get('href')

        listing = {
            'title': title,
            'price': price,
            'location': location,
            'image_url': picture_url,
            'url': url
        }
        data.append(listing)

    return pd.DataFrame(data)
    

In [ ]:
def get_ebay_listings(item):
    '''Returns a Pandas Data Frame with information about Ebay listings'''
    ebay_url = f"https://www.ebay.com/sch/i.html?_nkw={item}"
    response = requests.get(ebay_url)

    ebay_soup = soup(response.text, 'html.parser')
    
    item_divs = ebay_soup.find_all('li', class_='s-item s-item__pl-on-bottom')

    prices = [div.find('span', class_='s-item__price').text for div in item_divs]

    urls = [div.find('a', class_='s-item__link').get('href') for div in item_divs]

    
    image_divs = [div.find('div', class_='s-item__image-wrapper image-treatment') for div in item_divs]
    image_tags = [div.find('img') for div in image_divs]
    image_urls = [url.get('src') for url in image_tags]

    titles_div = [div.find('div', class_='s-item__title') for div in item_divs]
    titles = [div.find('span').text for div in titles_div]

    data = {
        'title': titles,
        'price': prices,
        'image_url': image_urls,
        'url': urls
    }

    return pd.DataFrame(data)

In [ ]:
def filter_ebay_listings(df):
    """Returns a cleaned data frame without any outliers"""
    # remove any special characters and convert to float
    df['price'] = df['price'].replace('[\$\,]', '', regex=True).astype(float)

    # remove the first two rows
    df = df.iloc[2:].reset_index(drop=True)

    # Remove outliers in the 'Price' column
    # Here we use the IQR method to identify outliers

    Q1 = df['price'].quantile(0.25)
    Q3 = df['price'].quantile(0.60)
    IQR = Q3 - Q1

    # Define lower and upper bound for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter out the outliers
    df_cleaned = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]

    return df_cleaned
    

In [ ]:
def calc_ebay_stats(df):
    """Returns a data frame with statistics of ebay listings"""
    # Calculate key statistics
    mean_price = df['price'].mean()
    median_price = df['price'].median()
    std_dev_price = df['price'].std()
    fair_price = median_price * 1.05  # Adjust by 5% for current market trends

    # store and return states in data frame
    stats = {
        'mean_price': [mean_price],
        'median_price': [median_price],
        'std_dev': [std_dev_price],
        'fair_price': [fair_price]
    }

    return pd.DataFrame(stats).round(2)


In [ ]:
def search_google_lens(image_url):
    """Returns Google Lens Data in the format of BeautifulSoup Object"""
    google_lens_url = f"https://lens.google.com/uploadbyurl?url={image_url}"
    browser = Browser('firefox')
    browser.visit(google_lens_url)
    
    time.sleep(1)
    # Scroll down to load more results
    
    # Define the number of times to scroll the page
    scroll_count = 2

    # Define the delay (in seconds) between each scroll
    scroll_delay = 2

    # Loop to perform scrolling
    for _ in range(scroll_count):
        # Execute JavaScript to scroll to the bottom of the page
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Pause for a moment to allow the content to load
        time.sleep(scroll_delay)
    
    # Parse the HTML
    html = browser.html

    # close browser window
    time.sleep(1)
    browser.quit()

    # Create a BeautifulSoup object from the scraped HTML
    return soup(html, 'html.parser')

In [ ]:
# specify the parameters
params = {
    'location': "pittsburgh",
    'item': "jersey",
    'condition': [],
    'min_price': 0,
    'max_price': 100,
    'days_since_listed': 10
}

# create the facebook marketplace query
url = create_facebook_marketplace_query(params)

# Visit the website and gather data
market_soup = fetch_marketplace_html(url)

In [ ]:
# parse html to create structured listing data
facebook_listings = parse_marketplace_data(market_soup)
facebook_listings

In [ ]:
ebay_listings = get_ebay_listings(facebook_listings['title'][0])
ebay_listings = filter_ebay_listings(ebay_listings)
ebay_listings

In [ ]:
ebay_stats = calc_ebay_stats(ebay_listings)
ebay_stats

In [ ]:
def upload_image(image_path):
    url = "https://lens.google.com/upload"
    headers = {
        'Content-Type': 'multipart/form-data',
    }

    with open(image_path, 'rb') as image_file:
        files = {
            'image': (image_path, image_file, 'image/jpeg')
        }
        
        response = requests.post(url, headers=headers, files=files)

    if response.status_code == 200:
        print('Image uploaded successfully')
        print('Response:', response.json())
    else:
        print('Failed to upload image')
        print('Status code:', response.status_code)
        print('Response:', response.text)
    return response.json()

# Replace 'your_image.jpg' with the path to your image file
# Replace 'your_oauth_token' with your actual OAuth token
response = upload_image('image.jpg')
response


In [ ]:
# Start a browser session (this example uses Chrome, but you can use Firefox by changing 'chrome' to 'firefox')
with Browser('firefox') as browser:
    # Open the URL where the form is located
    url = "https://www.google.com"
    browser.visit(url)
    time.sleep(3)
    # find the search by image button
    image_button = browser.find_by_css('nDcEnd')
    image_button.click()

    time.sleep(1)
    # Find the file input element and upload the image
    file_input = browser.find_by_name('encoded_image')  # Adjust the selector based on your form
    print(file_input)
    
    file_input.fill('image.jpg')
    time.sleep(3)
    # Optionally, fill other form fields if necessary
    # browser.find_by_name('other_field').fill('value')

    # Submit the form
    # submit_button = browser.find_by_name('submit')  # Adjust the selector based on your form
    # submit_button.click()

    # Optionally, wait for some response or next page to load
    browser.is_text_present('Success', wait_time=10)  # Adjust based on the expected success message

    # Print the URL of the current page (can be used to verify the navigation)
    print(browser.url)


In [ ]:
# NOTE: ONLY EBAY IMAGES ARE WORKING RIGHT NOW
# FACEBOOK IMAGES RETURN ERROR (Probably have bot auth.)
# MAYBE PASS THEM THROUGH A LINK GENERATOR FIRST

image_url = facebook_listings["image_url"][0]
print(image_url)
# image_filename = wget.download(image_url, "image.jpg")
image_url = "https://i.imgur.com/RQC5FHC.jpeg"
google_lens_soup = search_google_lens(image_url)